In [ ]:
import sys, os
pyStateSpace_path = os.path.abspath('../pyStateSpace')
try:
    import pystatespace
except ImportError:
    if pyStateSpace_path not in sys.path:
        sys.path.append(pyStateSpace_path)

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from copy import deepcopy
from multiprocessing import Pool
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # disable tensorflow warning messages

import numpy as np
import gym
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, make_scorer
import matplotlib.pyplot as plt
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines import PPO2
from tqdm.auto import tqdm, trange
from pystatespace import Trapezoid

from tanks import TanksFactory, TanksPhysicalEnv, TanksDataEnv, TanksDataRecurrentEnv
from plotting import plot_tanks
from utils import cache_function, cache_to_training_set, rewards_from_actions

SMALL_SIZE = 15
MEDIUM_SIZE = 17
BIGGER_SIZE = 19

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('lines', linewidth = 2.5)

## System

In [ ]:
n_tanks = 6
n_engines = 2
tstep = 1e0
seed = 0
np.random.seed(seed)
nominal_config = {
    'heights': np.ones(n_tanks),
    'cross_section':np.ones(n_tanks),
    'valves_min':np.zeros(n_tanks),
    'valves_max':np.ones(n_tanks),
    'resistances':np.ones(n_tanks) * 1e2,
    'pumps':np.ones(n_tanks) * 0.1,
    'engines':np.ones(n_engines) * 0.05
}

tanks = TanksFactory(n = n_tanks, e = n_engines, **nominal_config)
system = Trapezoid(dims=6, outputs=6, dx=tanks.dxdt, out=tanks.y, tstep=tstep)

In [ ]:
env_physical = DummyVecEnv([lambda: TanksPhysicalEnv(tanks, tstep=tstep)])
plot_tanks(env_physical.envs[0], plot='closed')
plt.suptitle('Fuel tank levels over time');

## System model

In [ ]:
# Random actions which are repeated for a random number
# of time steps
def rand_u(length: int, dim: int):
    i = 0
    u = np.zeros((length, dim))
    while i < arr_len:
        subseq = min(length - i, np.random.randint(1, int(length / 2)))
        u_ = np.random.choice(2, (1, dim))
        u[i:i+subseq] = u_
        i += subseq
    return u

In [ ]:
# Generate training data
episode_duration = sum(tanks.heights * tanks.cross_section) \
                   / min(sum(tanks.pumps), sum(tanks.engines))
episode_length = int(episode_duration / system.tstep)
episodes = 50

# An episode of length n will have n-1
# state[i], action[i], state[i+1] tuples
arr_len = episode_length - 1
Xtrain = np.empty((episodes * arr_len, n_tanks * 2))
Ytrain = np.empty((episodes * arr_len, n_tanks))
for e in range(episodes):
    u = rand_u(episode_length, n_tanks)
    t = np.linspace(0, episode_duration, num=episode_length, endpoint=False)
    x, _ = system.predict(t, tanks.heights, u)
    Xtrain[e * arr_len: (e+1) * arr_len, :n_tanks] = x[:-1]
    Xtrain[e * arr_len: (e+1) * arr_len, n_tanks:] = u[:-1]
    Ytrain[e * arr_len: (e+1) * arr_len] = x[1:]

print(Xtrain.shape, Ytrain.shape)

In [ ]:
grid = GridSearchCV(MLPRegressor(), scoring=make_scorer(r2_score, multioutput='uniform_average'),
                   param_grid={
                       'hidden_layer_sizes': ((64, 64), (128, 128), (256, 256), (512, 512)),
                       'activation': ('relu', 'logistic'),
                       'learning_rate_init': (1e-2, 5e-3, 1e-3),
                       'early_stopping': (True,)
                   },
                   n_jobs=12, verbose=1)
grid.fit(Xtrain, Ytrain)
pd.DataFrame(grid.cv_results_).sort_values(by='rank_test_score', ascending=True, axis=0).head()

In [ ]:
# Train on episodes-1, validate on 1 episode worth of instances:
est = grid.best_estimator_
est.set_params(random_state=seed)
# Plot performance
env_data = TanksDataEnv(tanks, est, tstep)
plot_tanks(env_data, plot='closed')
plt.suptitle('Modeled fuel tank levels over time');

## Degradation

In [ ]:
def degrade(tanks: TanksFactory, time: float, tfactor: np.ndarray,
            efactor: np.ndarray, **nominal):
    if not isinstance(tfactor, (list, tuple, np.ndarray)):
        # If a single degradation factor given, assume it is
        # identical for all tanks.
        pfactor = np.ones(n_tanks) * tfactor
    if not isinstance(efactor, (list, tuple, np.ndarray)):
        # If a single degradation factor given, assume it is
        # identical for all engines.
        efactor = np.ones(n_engines) * efactor
    for i in range(n_tanks):
        tanks.pumps[i] = nominal['pumps'][i] * (1 - time / tfactor[i])
        tanks.resistances[i] = nominal['resistances'][i] + \
                               nominal['resistances'][i] * time / tfactor[i]
    for i in range(n_engines):
        tanks.engines[i] = nominal['engines'][i] + \
                           nominal['engines'][i] * time / efactor[i]

## Environment

In [ ]:
# Reward function performance for normal vs degraded system
t = TanksFactory(n = n_tanks, e = n_engines, **nominal_config)
e = TanksPhysicalEnv(t, tstep)
tfactor = [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]
efactor = [20., np.inf]
plt.figure(figsize=(8, 4))

u, r, d = np.zeros(6), [], False
e.reset()
while not d:
    _,r_,d,_ = e.step(u)
    r.append(r_)
plt.plot(r, 'c-', label='Nominal, closed')

u, r, d = np.ones(6), [], False
e.reset()
while not d:
    _,r_,d,_ = e.step(u)
    r.append(r_)
plt.plot(r, 'c--', label='Nominal, open')

degrade(t, 10, tfactor, efactor, **nominal_config)
u, r, d = np.zeros(6), [], False
e.reset()
while not d:
    _,r_,d,_ = e.step(u)
    r.append(r_)
plt.plot(r, 'r:', label='Degraded, closed')

u, r, d = np.ones(6), [], False
e.reset()
while not d:
    _,r_,d,_ = e.step(u)
    r.append(r_)
plt.plot(r, 'r-.', label='Degraded, open')

plt.legend(framealpha=0.5, ncol=2)
plt.title('Rewards over single episode')
plt.ylabel('Reward')
plt.xlabel('Time /s')
plt.xlim(left=0, right=len(r))
plt.grid(True)
plt.show()

## Control Loop

1. Learn nominal policy from physical model
2. During operation/Online:
    1. Degrade system
    2. Improve control using on-policy RL on actual system
3. Offline (more sample efficiency):
    1. Learn data-driven model of system from recorded measurements
    2. Improve control using on-policy RL on data-driven model
4. Goto 2

In [ ]:
## Reinforcement learning
update_interval = 128
online_interval = 512
offline_interval = 2048
minibatch = 8  # number of batches per update_interval
policy_learning_rate = 1e-2
# policy_arch=[128, 128, dict(vf=[32, 32], pi=[32, 32])]
policy_arch=[64, 64, dict(vf=[16, 16], pi=[16, 16])]

In [ ]:
# A constructor that generates components for a single trial based
# on global variables
def get_components():
    env = DummyVecEnv([lambda: TanksPhysicalEnv(tanks, tstep=tstep)])
    model = deepcopy(grid.best_estimator_)
    model.set_params(warm_start=True)
    agent = PPO2(MlpPolicy, env, verbose=0, n_steps=update_interval,
                 nminibatches=minibatch, learning_rate=policy_learning_rate,
                 policy_kwargs=dict(net_arch=policy_arch), seed=seed)
    return env, model, agent

# A single trial which runs for periods*online_interval steps
# given a pair of degradation factors
def trial(tfactor, efactor, periods, agent, env, model, offline=True, return_caches=False):
    if return_caches:
        cache = []
    rewards_rl, rewards_open, rewards_closed = \
        np.zeros(periods), np.zeros(periods), np.zeros(periods)
    
    for period in trange(periods, leave=False):
        degrade(tanks, period, tfactor, efactor, **nominal_config)
        # Online-learning + control
        agent.set_env(env)
        x_cache, u_cache, d_cache, r_cache = [], [], [], []
        agent.learn(total_timesteps=online_interval,
                    callback=cache_function(x_cache, u_cache, d_cache, r_cache))
        # Accumulate rewards per period
        rewards_rl[period] = np.sum(r_cache) / online_interval
        rewards_open[period] = \
            rewards_from_actions(env.envs[0], np.ones((online_interval, n_tanks)))\
           / online_interval
        rewards_closed[period] = \
            rewards_from_actions(env.envs[0], np.zeros((online_interval, n_tanks)))\
           / online_interval
        if return_caches:
            cache.append((x_cache, u_cache, d_cache, r_cache))
        # Offline learning
        if offline:
            # Learn model
            xu, x_next = cache_to_training_set(x_cache, u_cache, d_cache)
            model.fit(xu, x_next)
            # Set data-driven environment
            agent.set_env(DummyVecEnv([lambda: TanksDataEnv(tanks, model, tstep)]))
            # Learn optimal control
            agent.learn(total_timesteps=offline_interval)
    
    if return_caches:
            return caches
    return rewards_rl, rewards_open, rewards_closed

# Run a trial multiple times where degradation factors are chosen randomly from
# a range. The number of degrading tanks is also random (<= atmost)
def multiple_trials(n_trials, periods=10, tfactors=(10, 20), efactors=(10, 20),
                    atmost_tanks=2, atmost_engines=2, offline=True):
    np.random.seed(seed)
    random = np.random.RandomState(seed)
    r_rl, r_open, r_closed = [], [], []

    for _ in trange(n_trials, leave=False):
        env, model, agent = get_components()
        
        tfactor = np.ones(n_tanks) * np.inf
        if atmost_tanks > 0:
            tanks_affected = random.randint(1, atmost_tanks + 1)
            idx_affected = random.choice(n_tanks, size=tanks_affected, replace=False)
            tfactor[idx_affected] = random.randint(*tfactors, size=tanks_affected)
        
        efactor = random.randint(*efactors, size=n_engines)
        if atmost_engines > 0:
            engines_affected = random.choice(1, atmost_engines + 1)
            idx_affected = random.choice(n_engines, size=engines_affected, replace=False)
            tfactor[idx_affected] = random.randint(*efactors, size=engines_affected)
        
        rewards_rl, rewards_open, rewards_closed = trial(tfactor, efactor, periods,
                                                         agent, env, model, offline)
        r_rl.append(rewards_rl)
        r_open.append(rewards_open)
        r_closed.append(rewards_closed)

    mean_rl = np.mean(r_rl, axis=0)
    mean_open = np.mean(r_open, axis=0)
    mean_closed = np.mean(r_closed, axis=0)
    std_rl = np.std(r_rl, axis=0)
    std_open = np.std(r_open, axis=0)
    std_closed = np.std(r_closed, axis=0)
    return mean_rl, mean_open, mean_closed,\
           std_rl, std_open, std_closed

In [ ]:
# Single trial, one fault
# Storing RL, valves open, valves closed rewards for when
# offline=True and False
offline_res_single = {True: (None, None, None), False: (None, None, None)}

# Plot single trial rewards for offline=True
periods = 10
tfactor = [np.inf, np.inf, np.inf, np.inf, np.inf, np.inf]
efactor = [20, np.inf]
offline = False
env, model, agent = get_components()

offline_res_single[offline] = trial(tfactor, efactor, periods,
                             agent, env, model, offline)

# Plot single trial rewards for offline=False
offline = True
env, model, agent = get_components()

offline_res_single[offline] = trial(tfactor, efactor, periods,
                             agent, env, model, offline)

In [ ]:
# Plot control actions for a single episode
period = 10
degrade(tanks, period, tfactor, efactor, **nominal_config)
plt.figure(figsize=(12, 8))
plot_tanks(env.envs[0], agent)
plt.suptitle('Fault in E1 at time={}s, offline={}'.format(period * online_interval, offline))
# plt.tight_layout();

In [ ]:
# Single trial, multiple faults
# Storing RL, valves open, valves closed rewards for when
# offline=True and False
offline_res_multiple = {True: (None, None, None), False: (None, None, None)}

# Plot single trial rewards for offline=True
periods = 10
tfactor = [np.inf, np.inf, np.inf, np.inf, 20, 20]
efactor = [20., np.inf]
offline = False
env, model, agent = get_components()

offline_res_multiple[offline] = trial(tfactor, efactor, periods,
                             agent, env, model, offline)

# Plot single trial rewards for offline=False
offline = True
env, model, agent = get_components()

offline_res_multiple[offline] = trial(tfactor, efactor, periods,
                             agent, env, model, offline)

In [ ]:
plt.figure(figsize=(8, 6))
plt.subplot(2,1,1)
if offline_res_single[True][1] is not None:
    rewards_open, rewards_closed = offline_res_single[True][1], offline_res_single[True][2]
else:
    rewards_open, rewards_closed = offline_res_single[False][1], offline_res_single[False][2]
rewards_rl_offline, rewards_rl_online = offline_res_single[True][0], offline_res_single[False][0]
plt.plot(rewards_open, '--', label='Open')
plt.plot(rewards_closed, ':', label='Closed')
if rewards_rl_offline is not None:
    plt.plot(rewards_rl_offline, 'g-')
if rewards_rl_online is not None:
    plt.plot(rewards_rl_online, 'g-.')

plt.legend()
plt.grid(True, which='both')
plt.ylabel('Rewards per step')
# plt.xlabel('Time /s')
plt.tick_params(labelbottom=False)
plt.xticks(plt.gca().get_xticks(), plt.gca().get_xticks() * online_interval)
plt.xlim(left=0, right=periods-1)
plt.title('Fault in E1')
# plt.show()

# plt.figure(figsize=(10, 8))
plt.subplot(2,1,2)
if offline_res[True][1] is not None:
    rewards_open, rewards_closed = offline_res_multiple[True][1], offline_res_multiple[True][2]
else:
    rewards_open, rewards_closed = offline_res_multiple[False][1], offline_res_multiple[False][2]
rewards_rl_offline, rewards_rl_online = offline_res_multiple[True][0], offline_res_multiple[False][0]
plt.plot(rewards_open, '--')
plt.plot(rewards_closed, ':')
if rewards_rl_offline is not None:
    plt.plot(rewards_rl_offline, 'g-', label='RL, Offline=True')
if rewards_rl_online is not None:
    plt.plot(rewards_rl_online, 'g-.', label='RL, Offline=False')

plt.legend()
plt.grid(True, which='both')
plt.ylabel('Rewards per step')
plt.xlabel('Time /s')
plt.xticks(plt.gca().get_xticks(), plt.gca().get_xticks() * online_interval)
plt.xlim(left=0, right=periods-1)
plt.title('Faults in T5, T6, E1')
plt.tight_layout()
plt.show()

In [ ]:
# Plot control actions for a single episode
period = 10
degrade(tanks, period, tfactor, efactor, **nominal_config)
plt.figure(figsize=(12, 8))
plot_tanks(env.envs[0], agent)
plt.suptitle('Faults in T5, T6, E1 at time={}s, offline={}'.format(period * online_interval, offline))
# plt.tight_layout();

In [ ]:
offline_res_agg = {True: (None,)*6, False: (None,)*6}

# Plot multiple trial rewards for offline=True
periods = 10
tfactors = (10, 30)
efactors = (10, 30)
atmost_tanks = 1
atmost_engines = 1
offline = True

offline_res_agg[offline] = \
    multiple_trials(20, periods, tfactors, efactors, atmost_tanks, atmost_engines, offline)

In [ ]:
# Plot multiple trial rewards for offline=False
offline = False

offline_res_agg[offline] = \
    multiple_trials(20, periods, tfactors, efactors, atmost_tanks, atmost_engines, offline)

In [ ]:
plt.figure(figsize=(8, 6))
rewards_open, rewards_closed = offline_res_agg[offline][1], offline_res_agg[offline][2]
std_open, std_closed = offline_res_agg[offline][4], offline_res_agg[offline][5]
rewards_rl_offline, rewards_rl_online = offline_res_agg[True][0], offline_res_agg[False][0]
std_rl_offline, std_rl_online = offline_res_agg[True][3], offline_res_agg[False][3]

plt.plot(rewards_open, '--', label='Open')
plt.plot(rewards_closed, ':', label='Closed')
if rewards_rl_offline is not None:
    plt.plot(rewards_rl_offline, 'g-', label='RL, Offline=True')
    plt.fill_between(np.arange(periods), rewards_rl_offline + std_rl_offline,
                     rewards_rl_offline - std_rl_offline,
                     alpha=0.3, color='g')
if rewards_rl_online is not None:
    plt.plot(rewards_rl_online, 'g-.', label='RL, Offline=False')
    plt.fill_between(np.arange(periods), rewards_rl_online + std_rl_online,
                     rewards_rl_online - std_rl_online,
                     alpha=0.3, color='g')
plt.gca().set_prop_cycle(None)
plt.fill_between(np.arange(periods), rewards_open + std_open, rewards_open - std_open, alpha=0.3)
plt.fill_between(np.arange(periods), rewards_closed + std_closed, rewards_closed - std_closed, alpha=0.3)
plt.legend(ncol=2)
plt.grid(True, which='both')
plt.ylabel('Rewards per step')
plt.xlabel('Time /s')
plt.xticks(plt.gca().get_xticks(), plt.gca().get_xticks() * online_interval)
plt.xlim(left=0, right=periods-1)
plt.title('Online')
plt.show()